# Match BGBM Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

explain_and_show_the_data = True
this_timestamp_for_data=20240312
# this_timestamp_for_data=time.strftime('%Y%m%d')

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    # "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    "data/wikidata_persons_botanists_20240312.csv", # with itemLabel + altLabel wyb, wye removed
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

if explain_and_show_the_data:
    pprint.pprint(wikidata.columns)
    display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wikidata_link', 'orcid_link',
       'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q100142069,Frida Eggens,NaN,NaN,Eggens,Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
1,http://www.wikidata.org/entity/Q100142069,Frida Eggens,Frida,F.,F. Eggens,Frida Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
2,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Elizabeth,E.,E. Harrison,Elizabeth Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
3,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,NaN,NaN,Mrs A. H.,Mrs A. H.,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
4,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Mrs Arnold,M. A.,M. A. Harrison,Mrs Arnold Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

if explain_and_show_the_data:
    display(wd_matchtest)
    display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"""The grandmother of female scientists in Ghana""",1
1,'Jan',1
2,'t. Hart,1
3,'t. Mannetje,1
4,('W.') S. W. Wong,1
...,...,...
159010,牧野富太郎(Tomitarô Makino),1
159011,百瀬静男(Sizuo Momose),1
159012,胡哲明,1


,canonical_string_fullname,item
,,count
0,"""The grandmother of female scientists in Ghana""",1
1,'Jan',1
2,'t Hart,1
3,'t Mannetje,1
4,('Wilson') Sze Wing Wong,1
...,...,...
184627,牧野富太郎(Tomitarô Makino),1
184628,百瀬静男(Sizuo Momose),1
184629,胡哲明,1


## Load Collectors Data Set

**Data sources:**

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
if explain_and_show_the_data: display(collectors)

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00,2008-06-05,2008-06-05
31620,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02
5,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
22504,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620,1997-07-10 16:00:00,1997-07-05,1997-07-14
22508,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682,1997-07-09 12:00:00,1997-07-07,1997-07-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Pignotti,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,5,https://herbarium.bgbm.org/object/B100513446,2010-06-13 00:00:00,2010-06-13,2010-06-13
12980,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,4,https://herbarium.bgbm.org/object/B100559866,2008-03-02 06:00:00,2004-06-03,2009-06-02
12941,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,7,https://herbarium.bgbm.org/object/B100810598,2015-07-07 00:00:00,2015-07-04,2015-07-09
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191,1989-07-12 00:00:00,1989-07-12,1989-07-12


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
if explain_and_show_the_data: 
    criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
    print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
    display(collectors[criterion_fullnames])

Show collecors given name has (propably) a full name (606 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2023,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00,1969-11-18,1969-11-18
22587,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00,1950-03-20,1950-03-20
17278,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00,1938-08-24,1938-08-24
15125,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00,1987-06-19,1987-06-19
399,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646,1987-07-03 00:00:00,1987-07-03,1987-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30642,Мирзоева,Mirzoeva,NaN,NaN,NaN,NaN,NaN,NaN,Mirzoeva [Мирзоева] & Gambarian & Pogosian,parsed:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Pog...,cleaned:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Po...,1,https://herbarium.bgbm.org/object/B100355321,1960-03-30 00:00:00,1960-03-30,1960-03-30
34734,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов],parsed:Popov Попов,cleaned:Popov Попов,3,https://dr.jacq.org/DR049416,1924-09-28 00:00:00,1923-06-17,1925-05-30
34735,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов] & Vvedensky,parsed:Popov Попов<SEP>Vvedensky,cleaned:Popov Попов<SEP>Vvedensky,4,https://herbarium.bgbm.org/object/B101149435,1923-08-27 06:00:00,1923-05-15,1924-05-22
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191,1989-07-12 00:00:00,1989-07-12,1989-07-12


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
if explain_and_show_the_data: 
    for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
        test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
        print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
        display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 603 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17278,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
347,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
3481,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
49605,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
13649,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 14 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13999,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
47886,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
13785,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
769,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
21795,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13850,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
if explain_and_show_the_data: print("combine parts of names similar to WikiData's given name labels")
collectors['canonical_string_collector_parsed'] = collectors[['given', 'particle', 'family', 'suffix']]\
    .fillna('')\
    .apply(
        lambda this_df: "{given}{particle}{family}{suffix}".format(
            given=this_df["given"],
            particle=" " + this_df["particle"] if this_df["particle"] else '', 
            family=" " + this_df["family"] if this_df["family"] else '', 
            suffix=", " + this_df["suffix"] if this_df["suffix"] else ''
        ).strip(), axis="columns"
    )

criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

if explain_and_show_the_data: 
    # display(collectors['canonical_string_collector_parsed'][criterion].head())
    display(collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10))

combine parts of names similar to WikiData's given name labels


,canonical_string_collector_parsed,particle
3481,M.L. Reyna de Aguilar,Reyna de
6,H. Abbas al Ani,Abbas al
39610,J. Francisco del Aquila,Francisco del
29014,A.L.M. Marcailhou d' Aymeric,Marcailhou d'
37779,P.R.O. Ritchie in Bally,Ritchie in
33712,H. Perrier de la Bathie,Perrier de la
33163,A.M.F.J. Palisot de Beauvois,Palisot de
49214,D. Van der Ben,Van der
53826,B. van den Boom,van den
53825,P. van den Boom,van den


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

if explain_and_show_the_data: display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
12973,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Pignotti,..."
12980,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,..."
12941,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian Գաբրիելյան,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich
47853,Թամանյան,Tamanian,NaN,NaN,NaN,NaN,NaN,NaN,Tamanian Թամանյան,Tamanian [Թամանյան; Таманян]


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)
collectors_unique.drop_duplicates(inplace=True)

if explain_and_show_the_data: display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a,1898-01-10 16:00:00,1897-11-01,1898-06-01
1,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",1,https://herbarium.bgbm.org/object/B100365313,1829-01-01 00:00:00,1829-01-01,1829-01-01
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Abaouz,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00,1997-07-05,1997-07-14
4,Acebey,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Acebey,"Acebey, A. (no. 563)",6,http://id.snsb.info/snsb/collection/719469/786...,1997-01-12 00:00:00,1992-11-28,2000-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18417,Оганесиан,М.,NaN,NaN,NaN,NaN,NaN,NaN,М. Оганесиан,"Oganesian,M. [Ոգանեսիան,Մ.; Оганесиан,М.], Ter...",25,https://herbarium.bgbm.org/object/B100275643,2005-01-18 17:06:00,1977-06-19,2010-06-11
18418,Androsov,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Androsov,"Androsov, Н. (no. 2404a)",1,http://id.snsb.info/snsb/collection/455833/556...,1911-05-06 00:00:00,1911-05-06,1911-05-06
18419,Габриелян,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Габриелян,"Gabrieljan,N. [Գաբրիելյան,Ն.; Габриелян,Н.] & ...",1,https://herbarium.bgbm.org/object/B100190831,1976-04-08 00:00:00,1976-04-08,1976-04-08
18420,Таманян,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Таманян,Tamanian [Թամանյան; Таманян],2,https://herbarium.bgbm.org/object/B100093359,1989-06-21 12:00:00,1986-07-31,1992-05-13


## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [9]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    """
    Get the matches of a data frame

    @param sparse_matrix: the matching matrix to apply
    @param A: the query data
    @param B: the match data
    @return: dataframe
    """
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [10]:
# some example data
if explain_and_show_the_data:
    print("Show ngram examples:")
    print("- simple name:", ngrams('Klazenga, N.'))
    print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
    print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
    print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[0]))
    
    # some example data
    for i, row in enumerate(range(5)):
        if (i == 0):
            print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:') 
        print("- {short_name} = {long_name}".format(
            short_name=wd_matchtest['canonical_string'].at[row],
            long_name=wd_matchtest_fullnames['canonical_string_fullname'].at[row]
        ))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['A A', ' A ', 'A D', ' Du', 'Du ', 'u T', ' Th', 'Tho', 'hou', 'oua', 'uar', 'ars']
- data from match-test: ['"Th', 'The', 'he ', 'e G', ' Gr', 'Gra', 'ran', 'and', 'ndm', 'dmo', 'mot', 'oth', 'the', 'her', 'er ', 'r O', ' Of', 'Of ', 'f F', ' Fe', 'Fem', 'ema', 'mal', 'ale', 'le ', 'e S', ' Sc', 'Sci', 'cie', 'ien', 'ent', 'nti', 'tis', 'ist', 'sts', 'ts ', 's I', ' In', 'In ', 'n G', ' Gh', 'Gha', 'han', 'ana', 'na"']
- data from match-test (full name): ['"Th', 'The', 'he ', 'e G', ' Gr', 'Gra', 'ran', 'and', 'ndm', 'dmo', 'mot', 'oth', 'the', 'her', 'er ', 'r O', ' Of', 'Of ', 'f F', ' Fe', 'Fem', 'ema', 'mal', 'ale', 'le ', 'e S', ' Sc', 'Sci', 'cie', 'ien', 'ent', 'nti', 'tis', 'ist', 'sts', 'ts ', 's I', ' In', 'In ', 'n G', ' Gh', 'Gha', 'han', 'ana', 'na"']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- "The grandmother of female sci

## Perform the Matching

In [11]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

print("Calculate rather the abbreviated names only …")
matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
if explain_and_show_the_data: display(matches)

Calculate rather the abbreviated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.138551950454712 s
Cossim matches calculated after 3.645404100418091 s
Get all matches together ...
Done. Matches calculated after 3.708197593688965 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,6274,G.U. Skinner,G. U. Skinner,1.0
1,11689,Martínez,Martínez,1.0
2,6749,H. Melchior,H. Melchior,1.0
3,14531,S. Stoyanov,S.Stoyanov,1.0
4,6751,H. Melzer,H.Melzer,1.0
...,...,...,...,...
16817,2256,C. Chumacero,Schumach.,0.5
16818,7139,H.G. van Eck,M.H.J.van Eck-Borsboom,0.5
16819,721,A. Rjosk,H. RJ,0.5
16820,7526,I. Amdursky,Gursky,0.5


In [12]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
print("Calculate rather the full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches_fullnames)

Calculate rather the full names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.268244981765747 s
Cossim matches calculated after 3.3229894638061523 s
Get all matches together ...
Done. Matches calculated after 3.3247852325439453 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,391,Émil Bureau,Émil Bureau,1.000
1,113,Ferdinand Weber,Ferdinand Weber,1.000
2,367,Walo Koch,Walo Koch,1.000
3,174,Henri Bernet,Henri Bernet,1.000
4,175,Henri Bordère,Henri Bordère,1.000
...,...,...,...,...
387,78,Degener Hatheway,Degen,0.515
388,137,Fundacion Cordes,Cordes,0.515
389,287,Oberst von Braun,Braun,0.510
390,172,Heinz Otto,Franz Otto Schmeil,0.507


## Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [13]:
if explain_and_show_the_data: print("join matches data frame back to source collectors dataframe")
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True) # drop all matches that the method gave a NA (we narrowed the similarity range from 0.5 to 1.0)
if explain_and_show_the_data:
    print("show match results of mostly abbreviated names")
    display(collectors_matches) # 17552 rows × 15 columns

join matches data frame back to source collectors dataframe
show match results of mostly abbreviated names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a,1898-01-10 16:00:00,1897-11-01,1898-06-01,0.0,A. A. da Silveira,A. A. d. Silveira,0.772
1,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",1,https://herbarium.bgbm.org/object/B100365313,1829-01-01 00:00:00,1829-01-01,1829-01-01,1.0,A. A. du Thouars,A. A. D. Thouars,0.719
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,2.0,A. Aaronsohn,A. Aaronsohn,1.000
4,Acebey,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Acebey,"Acebey, A. (no. 563)",6,http://id.snsb.info/snsb/collection/719469/786...,1997-01-12 00:00:00,1992-11-28,2000-04-03,3.0,A. Acebey,A. Acebey,1.000
5,Aceres,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aceres,"Álvarez de Zayas,A., Aceres,A., Bässler,M., Bi...",3,https://herbarium.bgbm.org/object/B100393538,1977-10-26 00:00:00,1977-10-25,1977-10-28,4.0,A. Aceres,G. Acerbi,0.617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18407,Čelakovský,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Čelakovský,Čelakovský,1,https://dr.jacq.org/DR049067,1870-07-09 00:00:00,1870-07-09,1870-07-09,16817.0,Čelakovský,L. J. Čelakovský,0.829
18408,Karjagin,İ.,NaN,NaN,NaN,NaN,NaN,NaN,İ. Karjagin,"Karjagin, İ.",1,https://herbarium.bgbm.org/object/B100601585,1934-02-01 00:00:00,1934-02-01,1934-02-01,16818.0,İ. Karjagin,Karjagin,1.000
18409,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Markova,M. & Černeva,Ž.",1,https://herbarium.bgbm.org/object/B100310826,1975-04-19 00:00:00,1975-04-19,1975-04-19,16819.0,Ž. Černeva,Korneva,0.621
18411,Fedorova,В.,NaN,NaN,NaN,NaN,NaN,NaN,В. Fedorova,"Tikhomirov,V. [Тихомиров,В.], Fedorova,T. & St...",1,https://herbarium.bgbm.org/object/B100630485,1991-06-04 00:00:00,1991-06-04,1991-06-04,16820.0,В. Fedorova,A. Fedorov,0.834


In [14]:
if explain_and_show_the_data: print("show full name matches, and append them to all matches")
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data:
    display(collectors_matches_fullname) # 429 rows × 15 columns

show full name matches, and append them to all matches


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Sauze,Abbé,NaN,NaN,NaN,NaN,NaN,NaN,Abbé Sauze,"Sauze,Abbé,J.",1,https://herbarium.bgbm.org/object/B100263049,1893-06-01 00:00:00,1893-06-01,1893-06-01,0,Abbé Sauze,Sauzé,0.572
1,Hassan,Abdisalam Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalam Sheikh Hassan,"Friis,I., Vollesen,K. & Abdisalam Sheikh Hassan",7,https://herbarium.bgbm.org/object/B100003700,1987-05-30 14:24:00,1987-05-27,1987-06-14,1,Abdisalam Sheikh Hassan,Abdisalam S. Hassan,0.758
2,Hassan,Abdisalem Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalem Sheikh Hassan,"Friis,I, Vollesen,K., & Abdisalem Sheikh Hassan",1,https://herbarium.bgbm.org/object/B100003663,1987-06-07 00:00:00,1987-06-07,1987-06-07,2,Abdisalem Sheikh Hassan,Abdisalam S. Hassan,0.617
3,García-Mendoza,Abisaí J.,NaN,NaN,NaN,NaN,NaN,NaN,Abisaí J. García-Mendoza,"García-Mendoza,Abisaí J. & Ramos,C.",2,https://herbarium.bgbm.org/object/B100242996,1984-05-30 00:00:00,1981-08-08,1987-03-22,3,Abisaí J. García-Mendoza,Abisai Josue García-Mendoza,0.704
4,Hussain,Abrar,NaN,NaN,NaN,NaN,NaN,NaN,Abrar Hussain,"Abedin,S. & Abrar Hussain",1,https://herbarium.bgbm.org/object/B100240364,1972-05-26 00:00:00,1972-05-26,1972-05-26,4,Abrar Hussain,Hussain,0.701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,Chang,Zhao Y.,NaN,NaN,NaN,NaN,NaN,NaN,Zhao Y. Chang,"Chang, Zhao Y. (no. 2004-449)",21,http://id.snsb.info/snsb/collection/469321/575...,2004-05-24 14:31:34,2004-05-08,2004-06-01,387,Zhao Y. Chang,Zhao Y.Chang,1.000
388,Liu,Zheng-yu,NaN,NaN,NaN,NaN,NaN,NaN,Zheng-yu Liu,"Liu,Zheng-yu",1,https://herbarium.bgbm.org/object/B100517349,1996-03-05 00:00:00,1996-03-05,1996-03-05,388,Zheng-yu Liu,Zheng Yu Liu,1.000
389,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,Zhou Zhuo,"Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08,2012-08-17,2012-09-02,389,Zhou Zhuo,Shou Zhou Zhang,0.717
390,Heldreich,von,NaN,NaN,NaN,NaN,NaN,NaN,von Heldreich,"Heldreich,von",12,https://dr.jacq.org/DR058755,1878-10-30 09:54:00,1855-05-25,1889-06-24,390,von Heldreich,I. von Heldreich,0.891


In [15]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
if explain_and_show_the_data:
    print("show match results of all abbreviated and full names")
    display(collectors_all_matches.head())

show match results of all abbreviated and full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,2.0,A. Aaronsohn,A. Aaronsohn,1.0
4803,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,E.C. Abbe,"Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,1946-05-22 12:00:00,1932-07-26,1960-03-18,4439.0,E.C. Abbe,E. C. Abbe,1.0
11277,Abbe,L.B.,NaN,NaN,NaN,NaN,NaN,NaN,L.B. Abbe,"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B.",1,https://herbarium.bgbm.org/object/BGT0003826,1960-03-18 00:00:00,1960-03-18,1960-03-18,10338.0,L.B. Abbe,L. B. Abbe,1.0
10003,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,J.R. Abbott,"Bécquer,E. & Abbott,J.R.",80,https://herbarium.bgbm.org/object/B100181131,1999-11-05 15:33:58,1992-05-20,2007-07-17,9191.0,J.R. Abbott,J.R.Abbott,1.0
17934,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,W.L. Abbott,"Abbott, W.L. (no. 1248)",4,http://id.snsb.info/snsb/collection/504820/626...,1921-08-04 16:00:00,1921-04-05,1922-04-27,16408.0,W.L. Abbott,W. L. Abbott,1.0


In [16]:
# Save the plain name matching results only ...
## cell split - code
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
## cell split - code
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [17]:
if explain_and_show_the_data: print("merge now the matching data and the wiki data’s on the conaonical string name")

collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

merge now the matching data and the wiki data’s on the conaonical string name


In [18]:
if explain_and_show_the_data:
    print("Old matching: Show example data of «Kotschy…» with Cosine Similiarity we had ~0.7 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)")
    print("Old matching: There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)")
    print("Old matching: There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/wiki/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")
    print("With itemLabel + altLabel: Apart from data given with family source name only, all matches seem to be correct, but interestingly the source of “C.G. Kotschy” was matched against “Kotschy” only")
    
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains('Kotschy')
    display(collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'itemLabel', 
        'canonical_string_fullname', # canonical_string_fullname contains the former itemMatchingLabel
        'wikidata_link',
    ]))

Old matching: Show example data of «Kotschy…» with Cosine Similiarity we had ~0.7 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)
Old matching: There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)
Old matching: There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/wiki/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/
With itemLabel + altLabel: Apart from data given with family source name only, all matches seem to be correct, but interestingly the source of “C.G. Kotschy” was matched against “Kotschy” only


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,canonical_string_fullname,wikidata_link
4789,1,http://id.snsb.info/snsb/collection/22980/3175...,C.G. Kotschy,Kotschy,0.815,Theodor Kotschy,Kotschy,http://www.wikidata.org/wiki/Q113299
4790,3,https://herbarium.bgbm.org/object/B100526350,Th Kotschy,Kotschy,0.778,Theodor Kotschy,Kotschy,http://www.wikidata.org/wiki/Q113299
4811,2215,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,1.000,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
4812,2215,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,1.000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
4813,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,1.000,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
4814,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,1.000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
13703,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,K. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
13704,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,Karl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
13705,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,Karl George Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
19194,294,http://id.snsb.info/snsb/collection/117808/176...,T. Kotschy,T. Kotschy,1.000,Theodor Kotschy,T. Kotschy,http://www.wikidata.org/wiki/Q113299


In [19]:
if explain_and_show_the_data:
    print("list all column names so far, and show example data")
    pprint.pprint(collectors_matches_g1_merged_wikidata.columns)
    display(collectors_matches_g1_merged_wikidata.head())

list all column names so far, and show example data
Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",...,9650-1,Silveira,NaN,1867,1945,http://www.wikidata.org/wiki/Q19002102,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/9650-1,NaN
1,Silveira,A.A.,NaN,da,NaN,NaN,NaN,NaN,A.A. da Silveira,"da Silveira,A.A.",...,9650-1,Silveira,NaN,1867,1945,http://www.wikidata.org/wiki/Q19002102,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/9650-1,NaN
2,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",...,10638-1,A.Thouars,Q2821491,1793,1864,http://www.wikidata.org/wiki/Q2821491,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/10638-1,https://bionomia.net/Q2821491
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",...,23-1,Aarons.,Q2086130,1876,1919,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130
4,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",...,23-1,Aarons.,Q2086130,1876,1919,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130


In [20]:
# this_timestamp_for_data=20231116
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    # Select useful columns for data results
    collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
        ['canonical_string_collector_parsed', 'family', 'given', 
         'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
         'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 
         # 'wyb'
        ]
    ].copy()
    
    # Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
    collectors_wikidata_cossimOrKmeans.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
    
    display(collectors_wikidata_cossimOrKmeans)
    # cell split - code
    # Kotschy example again with all merged columns
    pd.set_option("display.max_columns", None) # default ?20
    
    criterion = collectors_wikidata_cossimOrKmeans['canonical_string_collector_parsed'].str.contains('Kotschy')
    print("old data without altLabel: Show example of «Kotschy…» with similarities of 0.5 … 1.0")
    print("old data without altLabel: There was a match of Kotschy, C.G.T. → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 and it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")
    display(collectors_wikidata_cossimOrKmeans[criterion])
    # cell split - code

    # TODO further evaluation or filtering, counting, clean up aso.
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # bgbm_collectors_cosine-similarity_wikidata-botanists_%s.csv
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_wikidata_cossimOrKmeans.to_csv(this_output_file)
    
    print("Wrote matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )
    pd.set_option("display.max_columns", 20)

## Write DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

### Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [21]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [22]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    [
        'canonical_string_collector_parsed', 'family', 'given', 
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
        'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod',
        # 'wyb'
    ]
].copy()

# order by canonical_string_collector_parsed (actual collector name) (asc)
#   order by similarity (desc) or namematch_distance (asc), 
#     order by number of Wikidata items (asc) and 
#       order by number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    by=['canonical_string_collector_parsed', 'namematch_similarity', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossimOrKmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy().drop_duplicates(ignore_index=True)

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime_data"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime_data"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime_data"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime_data"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime_data"
] = 0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = -1
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime_data"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime_data"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_data_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_data_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_data_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_data_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_data_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_data_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_data_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime_data"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime_data"],
    lifetime_score_annote=row["custom_score_lifetime_data_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    # by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    # ascending=[False, True, True, False], 
    by=['canonical_string_collector_parsed', 'custom_score_overall', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
if explain_and_show_the_data:
    print("example data: names having year of birth (yob) < eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    print("example data: names having year of birth (yob) > eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))

example data: names having year of birth (yob) < eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
0,A. A. da Silveira,Alvaro Astolpho da Silveira,0.386,0.772 (cos similarity); 0.39 (score overall); ...,0.0,0.772,1867-1945,1897-11-01,1898-06-01,True,True,1.0,full match
1,A. A. du Thouars,Abel Aubert Dupetit Thouars,0.360,0.719 (cos similarity); 0.36 (score overall); ...,0.0,0.719,1793-1864,1829-01-01,1829-01-01,True,True,1.0,full match
2,A. Aaronsohn,Aaron Aaronsohn,0.500,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.000,1876-1919,1908-06-20,1908-06-20,True,True,1.0,full match
3,A. Acebey,Amparo Acebey,0.500,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.000,1973-?,1992-11-28,2000-04-03,True,<NA>,1.0,OK? year of death is missing
4,A. Aceres,Giuseppe Acerbi,0.154,0.617 (cos similarity); 0.15 (score overall); ...,0.0,0.617,1773-1846,1977-10-25,1977-10-28,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."


example data: names having year of birth (yob) > eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
24,A. Andersson,Axel Andersson,-0.500,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.000,1895-?,1891-01-01,1981-08-29,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
25,A. Andersson,I. Anita Andersson,-0.500,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.000,1959-?,1891-01-01,1981-08-29,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
26,A. Angus,Put Ang,-0.133,0.532 (cos similarity); -0.13 (score overall);...,0.0,0.532,1955-?,1952-11-30,1963-06-09,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
34,A. Araujo-Murakami,Alejandro Araujo-Murakami,-0.250,1.0 (cos similarity); -0.25 (score overall); 2...,0.0,1.000,2010-?,2005-03-08,2005-06-10,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
37,A. Artzt,Franz Friedrich Anton Artzt,-0.441,0.882 (cos similarity); -0.44 (score overall);...,0.0,0.882,1844-1922,1807-06-01,1927-08-01,False,False,-1.0,life time not matching any enventDate (yob + 1...


In [23]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

if explain_and_show_the_data:
    print("the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …")
    display(dwcagent_attr_output.head(20))

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …


,occurrenceID,alternateName,displayOrder,family,given,custom_namematch_similarity,verbatimName,name,identifier,action,...,yod,namematch_similarity_annotation,life_time_periode,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation,custom_score_multiple_names,custom_score_overall,attributionRemarks
0,https://herbarium.bgbm.org/object/B200129406a,A. A. da Silveira,1,Silveira,A. A. da,0.772,"Silveira,A.A. da",Alvaro Astolpho da Silveira,http://www.wikidata.org/entity/Q19002102,collected,...,1945,0.772 (cos similarity),1867-1945,True,True,1.0,full match,0.0,0.386,0.772 (cos similarity); 0.39 (score overall); ...
1,https://herbarium.bgbm.org/object/B100365313,A. A. du Thouars,1,Thouars,A. A. du,0.719,"Thouars,A. A.du Petit-",Abel Aubert Dupetit Thouars,http://www.wikidata.org/entity/Q2821491,collected,...,1864,0.719 (cos similarity),1793-1864,True,True,1.0,full match,0.0,0.360,0.719 (cos similarity); 0.36 (score overall); ...
2,https://herbarium.bgbm.org/object/B100379341,A. Aaronsohn,1,Aaronsohn,A.,1.000,"Aaronsohn,A.",Aaron Aaronsohn,http://www.wikidata.org/entity/Q2086130,collected,...,1919,1.0 (cos similarity),1876-1919,True,True,1.0,full match,0.0,0.500,1.0 (cos similarity); 0.50 (score overall); 18...
3,http://id.snsb.info/snsb/collection/719469/786...,A. Acebey,1,Acebey,A.,1.000,"Acebey, A. (no. 563)",Amparo Acebey,http://www.wikidata.org/entity/Q5673219,collected,...,<NA>,1.0 (cos similarity),1973-?,True,<NA>,1.0,OK? year of death is missing,0.0,0.500,1.0 (cos similarity); 0.50 (score overall); 19...
4,https://herbarium.bgbm.org/object/B100393538,A. Aceres,1,Aceres,A.,0.617,"Álvarez de Zayas,A., Aceres,A., Bässler,M., Bi...",Giuseppe Acerbi,http://www.wikidata.org/entity/Q55007624,collected,...,1846,0.617 (cos similarity),1773-1846,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.154,0.617 (cos similarity); 0.15 (score overall); ...
5,https://herbarium.bgbm.org/object/B100092853,A. Ackermann,1,Ackermann,A.,0.914,"Weigend,M., Ackermann,A. & Castillo,J.A.",Manfred Ackermann,http://www.wikidata.org/entity/Q47112660,collected,...,<NA>,0.914 (cos similarity),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.914 (cos similarity); 0.00 (score overall); ...
6,https://herbarium.bgbm.org/object/B100629441,A. Acosta,1,Acosta,A.,0.893,"Zardini,E.M. & Acosta,A.",Cristóbal Acosta,http://www.wikidata.org/entity/Q384460,collected,...,1594,0.893 (cos similarity),1515-1594,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.223,0.893 (cos similarity); 0.22 (score overall); ...
7,http://id.snsb.info/snsb/collection/513255/635...,A. Ade,1,Ade,A.,1.000,"Ade, A.",Alfred Ade,http://www.wikidata.org/entity/Q8195254,collected,...,1968,1.0 (cos similarity),1876-1968,True,True,1.0,full match,0.0,0.500,1.0 (cos similarity); 0.50 (score overall); 18...
8,https://herbarium.bgbm.org/object/B101010034,A. Adebowale,1,Adebowale,A.,0.571,"Balkwill,K., McCallum,D.A., Mbatha,H.P., Adebo...",Alfred Ade,http://www.wikidata.org/entity/Q8195254,collected,...,1968,0.571 (cos similarity),1876-1968,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.143,0.571 (cos similarity); 0.14 (score overall); ...
9,https://herbarium.bgbm.org/object/B100068937,A. Afzelius,1,Afzelius,A.,1.000,"Afzelius,A. & Daniell,W.F.",Adam Afzelius,http://www.wikidata.org/entity/Q347903,collected,...,1837,1.0 (cos similarity),1750-1837,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.250,1.0 (cos similarity); 0.25 (score overall); 17...


In [24]:
if explain_and_show_the_data:
    print("show column-reduced examples of ?multiple name cases …")
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda this_score: this_score < 0 )  # show multiple name cases
    
    # display(dwcagent_attr_output[criterion].head(20))
    display(dwcagent_attr_output[criterion].drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))

show column-reduced examples of ?multiple name cases …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime_data
15,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,1,Anatolij Aleksandrovič Ničiporovič,0.625 (cos similarity); 0.00 (score overall); ...,2015-07-04,2015-07-09,http://www.wikidata.org/entity/Q26244283,0.000,0.625,-0.5,0.5
16,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,2,A.A. Richter,0.625 (cos similarity); 0.00 (score overall); ...,2015-07-04,2015-07-09,http://www.wikidata.org/entity/Q4394909,0.000,0.625,-0.5,0.5
24,https://herbarium.bgbm.org/object/B100002495,"Andersson,A.",A. Andersson,1,Axel Andersson,1.0 (cos similarity); -0.50 (score overall); 1...,1891-01-01,1981-08-29,http://www.wikidata.org/entity/Q123652899,-0.500,1.000,-0.5,-0.5
25,https://herbarium.bgbm.org/object/B100002495,"Andersson,A.",A. Andersson,2,I. Anita Andersson,1.0 (cos similarity); -0.50 (score overall); 1...,1891-01-01,1981-08-29,http://www.wikidata.org/entity/Q21505194,-0.500,1.000,-0.5,-0.5
53,http://id.snsb.info/snsb/collection/463093/564...,"Barmin, A.",A. Barmin,1,Eugenius Warming,0.578 (cos similarity); 0.00 (score overall); ...,1990-08-20,1990-08-20,http://www.wikidata.org/entity/Q355888,0.000,0.578,-0.5,0.5
54,http://id.snsb.info/snsb/collection/463093/564...,"Barmin, A.",A. Barmin,2,Eugenius Warming,0.578 (cos similarity); 0.00 (score overall); ...,1990-08-20,1990-08-20,http://www.wikidata.org/entity/Q355888,0.000,0.578,-0.5,0.5
66,https://je.jacq.org/JE04009579,"Bellamy,A.",A. Bellamy,1,Charles Lawrence Bellamy,0.895 (cos similarity); 0.22 (score overall); ...,1981-01-27,1984-06-27,http://www.wikidata.org/entity/Q19115006,0.224,0.895,-0.5,1.0
67,https://je.jacq.org/JE04009579,"Bellamy,A.",A. Bellamy,2,R.E. Bellamy,0.895 (cos similarity); -0.22 (score overall);...,1981-01-27,1984-06-27,http://www.wikidata.org/entity/Q36658695,-0.224,0.895,-0.5,0.0
77,http://id.snsb.info/snsb/collection/661051/718...,"Bertoloni, A.",A. Bertoloni,1,Antonio Bertoloni,1.0 (cos similarity); 0.25 (score overall); 17...,1809-05-31,1809-05-31,http://www.wikidata.org/entity/Q599936,0.250,1.000,-0.5,1.0
76,http://id.snsb.info/snsb/collection/661051/718...,"Bertoloni, A.",A. Bertoloni,2,Antonio Bertoloni Jr.,1.0 (cos similarity); 0.00 (score overall); 18...,1809-05-31,1809-05-31,http://www.wikidata.org/entity/Q120469486,0.000,1.000,-0.5,0.5


In [25]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231116
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20240312.csv (6698 kB)


## Documentation

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label (perhaps similar to the full name)
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))